**Optimization and Thresholding using RBFN:** The code optimizes the frame scores using a Radial Basis Function Network (RBFN). It applies K-Means clustering, and performs an RBF transformation using a Gaussian function to optimize the scores. It also determines the threshold for training data.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from scipy.spatial.distance import cdist
from numpy.linalg import lstsq

# Load frame scores
file_path = "frame_scores_train.xlsx"  # Update with your file path
df = pd.read_excel(file_path)

# Extract frame scores
frame_scores = df["Fs"].values.reshape(-1, 1)

# Apply K-Means clustering to find cluster centers (q = 3)
kmeans = KMeans(n_clusters=3, random_state=42, n_init=10)
kmeans.fit(frame_scores)
centers = kmeans.cluster_centers_

# RBF transformation using Gaussian function
def rbf_transform(x, centers, sigma=10):
    return np.exp(-cdist(x, centers, 'sqeuclidean') / (2 * sigma ** 2))

# Apply RBF transformation
phi = rbf_transform(frame_scores, centers)

# Solve for synaptic weights W using least squares optimization
desired_output = frame_scores  # Use original frame scores as targets
W, _, _, _ = lstsq(phi, desired_output, rcond=None)

# Compute optimized frame scores using Fs_opti = sum(W_k * phi(f))
optimized_frame_scores = phi @ W

# Save results to DataFrame
df_results = df.copy()
df_results["optimized_frame_scores"] = optimized_frame_scores

# Compute the threshold from the maximum optimized frame score
Fs_th = np.max(optimized_frame_scores)
df_results["Fs_th"] = Fs_th

# Save results in the same Excel file
with pd.ExcelWriter(file_path, engine="openpyxl", mode="a", if_sheet_exists="replace") as writer:
    df_results.to_excel(writer, sheet_name="optimized_frame_scores", index=False)
    pd.DataFrame({"Threshold": [Fs_th]}).to_excel(writer, sheet_name="threshold_value", index=False)

print("RBFN optimization complete. Results saved.")
